In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm


project_path = "/home/wangshu/rag/hier_graph_rag"
sys.path.append(os.path.abspath(project_path))


direct_res_path = "/mnt/data/wangshu/hcarag/mintaka/hcarag/hc_index_8b/qa/global_10_20_20_15_mr_QA_.csv"
direct_df = pd.read_csv(direct_res_path)
entity_df_path = "/mnt/data/wangshu/hcarag/mintaka/hcarag/hc_index_8b/entity_df_index.csv"
entity_df = pd.read_csv(entity_df_path)
community_df_path = "/mnt/data/wangshu/hcarag/mintaka/hcarag/hc_index_8b/community_df_index.csv"
community_df = pd.read_csv(community_df_path)

direct_df.head(2)

,question,answer_raw,answers,label,id,raw_result,pred,acc,hit,f1,precision,recall
0,What man was a famous American author and also...,"{'answerType': 'entity', 'answer': [{'name': '...",Mark Twain,mark twain,0,---Direct Answer---: Mark Twain\n---Brief Anal...,Mark Twain,1.0,1,0.4,0.25,1.0
1,How many Academy Awards has Jake Gyllenhaal be...,"{'answerType': 'numerical', 'answer': [1], 'me...",1,1,1,---Direct Answer---: 0 | 0\n---Brief Analysis-...,0 | 0,0.0,0,0.0,0.00,0.0


In [6]:
entity_df["node_description"] = entity_df["name"] + " " + entity_df["description"]
entity_df_description = entity_df["node_description"].values
entity_df_description = " ".join(entity_df_description)
print(entity_df_description[0:100])

Diana Ross Diana Ross (born March 26, 1944) is an American singer and actress. She was the lead sing


In [17]:
community_df["community_desc"] = community_df["summary"] + " " + community_df["title"]
c_df_desc = community_df["community_desc"].values
community_content = " ".join([c for c in c_df_desc if type(c) == str])
print(community_content[0:100])

The community revolves around the Green Valley Mall, which is a central location for various events 


In [21]:
text_length = []
for _, row in community_df.iterrows():
    if type(row["community_desc"]) is str:
        text_length.append(len(row["community_desc"]))

community_mean_text_length = sum(text_length) / len(text_length)
print(community_mean_text_length)

text_length_entity = []
for _, row in entity_df.iterrows():
    text_length_entity.append(len(row["node_description"]))
entity_mean_text_length = sum(text_length_entity) / len(text_length_entity)
print(entity_mean_text_length)


278.10692498491653
349.5613034229356


In [25]:
cnt_entity = 0
cnt_community = 0
cnt_both = 0

for _, row in tqdm(direct_df.iterrows()):
    answer = row["label"]
    
    if answer in entity_df_description:
        cnt_entity += 1
    
    if answer in community_content:
        cnt_community += 1
    
    if answer in entity_df_description or answer in community_content:
        cnt_both += 1
        
print(f"number of question: {len(direct_df)}")
    
print(f"entity ratio: {cnt_entity / len(direct_df)}")
print(f"community ratio: {cnt_community / len(direct_df)}")
print(f"both answer ratio: {cnt_both / len(direct_df)}")


4000it [03:04, 21.66it/s] 

number of question: 4000
entity ratio: 0.431
community ratio: 0.37775
both answer ratio: 0.431


In [2]:
path_qa_df_ori = "/mnt/data/wangshu/hcarag/mintaka/data/mintaka_test.json"
qa_df_ori = pd.read_json(path_qa_df_ori)

qa_df_ori["answer_label"] = qa_df_ori["answer"].apply(lambda x: x["mention"])
qa_df_ori = qa_df_ori[["id", "question", "answer_label", "category", "complexityType","answer"]]
sel_comples = [
    "superlative",
    "ordinal",
    "multihop",
    "intersection",
    "difference",
    "generic",
]

print(qa_df_ori["complexityType"].unique())
sel_qa_df = qa_df_ori[qa_df_ori["complexityType"].isin(sel_comples)]
print(sel_qa_df.shape)

sel_qa_df.head(2)

['intersection' 'count' 'comparative' 'yesno' 'generic' 'ordinal'
 'multihop' 'difference' 'superlative']
(2800, 6)


,id,question,answer_label,category,complexityType,answer
0,fae46b21,What man was a famous American author and also...,Mark Twain,history,intersection,"{'answerType': 'entity', 'answer': [{'name': '..."
5,b41ae115,Who performed at the Super Bowl XXIII halftime...,Elvis Presto,sports,generic,"{'answerType': 'entity', 'answer': None, 'ment..."


In [38]:
sel_def = qa_df_ori[qa_df_ori['complexityType']== "difference"]
sel_def.head(2)

,id,question,answer_label,category,complexityType
76,61216214,Which 2000 Christopher Nolan movie does not st...,Memento,movies,difference
77,c3f408f4,Which 2020 Christopher Nolan movie does not st...,Tenet,movies,difference


In [5]:
cnt_gr = 0

direct_df['answers_lists'] = direct_df['answers'].apply(lambda x: eval(x))
gr_base_path = "/mnt/data/wangshu/hcarag/FB15k/KG/cached_desc/"

for idx, row in tqdm(direct_df.iterrows()):
    answer = row["answers_lists"]
    with open(gr_base_path + str(idx) + ".txt", "r") as f:
        gr_text = f.read()
    for ans in answer:
        if ans in gr_text:
            cnt_gr += 1
            break
    
print(cnt_gr) 
print(cnt_gr / len(direct_df))
print(len(direct_df))

0it [00:00, ?it/s]

2032it [00:01, 1668.27it/s]

515
0.2534448818897638
2032
